In [43]:
import pandas as pd
import numpy as np


In [44]:
df = pd.read_csv("interview_qna_scoring_dataset.csv")
df.head()

,Question,Answers,Answer_Score,Explanation
0,Describe who you are?,I am a dedicated professional with five years ...,100,This answer is highly effective because it pro...
1,Describe who you are?,Just someone trying to find my way in the world.,10,"This answer is vague, offering little insight ..."
2,Describe who you are?,"I thrive in dynamic environments, aiming to so...",90,The response effectively highlights the indivi...
3,Describe who you are?,"I exist, I guess.",1,This answer is the least effective as it provi...
4,Describe who you are?,An adaptable professional excelling in high-pr...,80,"This answer is effective, underlining adaptabi..."


In [45]:
x=df["Question"]+" "+df["Answers"]+df["Explanation"]
y=df["Answer_Score"]

In [46]:
# Convert text data to TF-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(x)

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_tfidf,y,test_size=0.2,random_state=0)

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [68]:
# Fit the models
linear = LinearRegression()
linear.fit(x_train, y_train)

log = LogisticRegression()
log.fit(x_train, y_train)

svr = SVR()
svr.fit(x_train, y_train)

dtr = DecisionTreeRegressor()
dtr.fit(x_train, y_train)

rf = RandomForestRegressor(n_estimators=500, criterion="squared_error", random_state=0)
rf.fit(x_train, y_train)



RandomForestRegressor(n_estimators=500, random_state=0)

In [ ]:
# Make predictions
predict1 = linear.predict(x_test)
predict2 = log.predict(x_test)
predict3 = svr.predict(x_test)
predict4 = dtr.predict(x_test)
predict5 = rf.predict(x_test)

# Evaluate predictions using regression metrics
metrics = {
    'Model': ['Linear Regression', 'Logistic Regression', 'SVR', 'Decision Tree Regressor', 'Random Forest Regressor'],
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R²': []
}

for predict in [predict1, predict2, predict3, predict4, predict5]:
    metrics['MAE'].append(mean_absolute_error(y_test, predict))
    metrics['MSE'].append(mean_squared_error(y_test, predict))
    metrics['RMSE'].append(mean_squared_error(y_test, predict, squared=False))
    metrics['R²'].append(r2_score(y_test, predict))

# Convert metrics dictionary to DataFrame for better readability
metrics_df = pd.DataFrame(metrics)
print(metrics_df)


In [63]:
import pickle

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf, file)

# Save the trained Logistic Regression model
with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(lg, file)


In [64]:
# Assuming the trained TF-IDF vectorizer and Logistic Regression model are saved
import pickle
import numpy as np

# Load the trained TF-IDF vectorizer and model
with open('tfidf_vectorizer.pkl', 'rb') as file:
    tfidf = pickle.load(file)

with open('logistic_regression_model.pkl', 'rb') as file:
    model = pickle.load(file)

def predict_score(question, answer):
    # Combine the question and answer as done during training
    input_text = question + " " + answer
    
    # Transform the input text using the trained TF-IDF vectorizer
    input_features = tfidf.transform([input_text])
    
    # Predict the score using the trained model
    predicted_score = model.predict(input_features)
    
    return predicted_score[0]

# Example usage
question = "Describe who you are?"
answer = "I am a dedicated professional with five years of experience"
predicted_score = predict_score(question, answer)
print(f"Predicted Score: {predicted_score}")



Predicted Score: 80
